In [1]:
import requests

def read_uniprot_fasta_url(url: str) -> str:
    response = requests.get(url, timeout=30)
    response.raise_for_status()

    lines = response.text.splitlines()
    sequence = "".join(line.strip() for line in lines if not line.startswith(">"))
    return sequence

sequence = read_uniprot_fasta_url("https://rest.uniprot.org/uniprotkb/P04637.fasta")

hydrophobic = {'A','V','I','L','M','F','Y','W'}

def score(seq):
    return sum(2 if r in hydrophobic else 1 for r in seq)

def leaderboard_consecutive(sequence, k):
    leaderboard = [""]  # start empty
    best_score = 0
    best_seq = ""

    # Step-by-step expansion (like leaderboard)
    for step in range(1, k+1):
        new_board = []
        for cand in leaderboard:
            start_idx = len(cand)
            if start_idx < len(sequence):
                # only take next consecutive character
                new_board.append(cand + sequence[start_idx])
        leaderboard = new_board

        # Update best score and motif
        for cand in leaderboard:
            sc = score(cand)
            if sc > best_score:
                best_score = sc
                best_seq = cand

    # Collect all top motifs with same best score
    top_motifs = []
    for i in range(len(sequence) - k + 1):
        subseq = sequence[i:i+k]
        if score(subseq) == best_score:
            top_motifs.append((subseq, best_score))

    return best_seq, best_score, top_motifs

# Run for k=5
best, best_sc, top_list = leaderboard_consecutive(sequence, k=5)

print("Best consecutive motif:", best)
print("Best score:", best_sc)

print("\nTop Motifs with highest score:")
for motif, sc in top_list:
    print(motif, sc)

Best consecutive motif: MEEPQ
Best score: 6

Top Motifs with highest score:
MEEPQ 6
SDPSV 6
DPSVE 6
PSVEP 6
SVEPP 6
EPPLS 6
PPLSQ 6
PLSQE 6
LSQET 6
SQETF 6
QETFS 6
ETFSD 6
LPENN 6
PENNV 6
SPLPS 6
PLPSQ 6
LSPDD 6
SPDDI 6
PDDIE 6
DDIEQ 6
FTEDP 6
GPDEA 6
PDEAP 6
DEAPR 6
PRMPE 6
PLSSS 6
SSSVP 6
SSVPS 6
SVPSQ 6
VPSQK 6
SQKTY 6
QKTYQ 6
KTYQG 6
TYQGS 6
QGSYG 6
LHSGT 6
HSGTA 6
SGTAK 6
GTAKS 6
KSVTC 6
SVTCT 6
TCTYS 6
CTYSP 6
AKTCP 6
KTCPV 6
TCPVQ 6
VDSTP 6
PGTRV 6
GTRVR 6
YKQSQ 6
QSQHM 6
SQHMT 6
QHMTE 6
VRRCP 6
DSDGL 6
APPQH 6
PPQHL 6
RVEGN 6
EGNLR 6
LDDRN 6
DRNTF 6
RNTFR 6
NTFRH 6
TFRHS 6
PYEPP 6
YEPPE 6
EPPEV 6
PPEVG 6
PEVGS 6
EVGSD 6
VGSDC 6
DCTTI 6
CTTIH 6
MCNSS 6
NSSCM 6
SSCMG 6
SCMGG 6
GGMNR 6
GMNRR 6
MNRRP 6
NRRPI 6
TLEDS 6
LEDSS 6
SSGNL 6
LGRNS 6
GRNSF 6
RNSFE 6
CACPG 6
ACPGR 6
EEENL 6
EENLR 6
ENLRK 6
NLRKK 6
LRKKG 6
PHHEL 6
HHELP 6
HELPP 6
ELPPG 6
LPPGS 6
STKRA 6
LPNNT 6
KKKPL 6
KKPLD 6
KPLDG 6
PLDGE 6
QIRGR 6
IRGRE 6
GRERF 6
RERFE 6
RELNE 6
QAGKE 6
AGKEP 6
GGSRA 6
GSRAH 6
SRAHS 6
RAHS